In [2]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import datetime
import time

In [3]:
NORS_df = pd.read_csv('../data/NORS.csv')

C:\Users\rayna\AppData\Local\Temp\ipykernel_11340\874844664.py:1: DtypeWarning: Columns (8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  NORS_df = pd.read_csv('../data/NORS.csv')


In [4]:
NORS_df.head()

,Year,Month,State,Primary Mode,Etiology,Serotype or Genotype,Etiology Status,Setting,Illnesses,Hospitalizations,Info On Hospitalizations,Deaths,Info On Deaths,Food Vehicle,Food Contaminated Ingredient,IFSAC Category,Water Exposure,Water Type,Animal Type
0,1971,2,California,Water,Copper,NaN,Confirmed,Restaurant,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
1,1971,6,Arkansas,Water,Hepatitis A,NaN,Confirmed,Store,98,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Other,NaN
2,1971,6,Missouri,Water,Unknown,NaN,Suspected,Subdivision/Neighborhood,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
3,1971,6,Alabama,Water,Selenium,NaN,Confirmed,Unknown,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Individual/Private,NaN
4,1971,6,Vermont,Water,Unknown,NaN,Suspected,Community/municipality,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN


In [5]:
NORS_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66713 entries, 0 to 66712
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Year                          66713 non-null  int64  
 1   Month                         66713 non-null  int64  
 2   State                         66713 non-null  object 
 3   Primary Mode                  66713 non-null  object 
 4   Etiology                      50375 non-null  object 
 5   Serotype or Genotype          16470 non-null  object 
 6   Etiology Status               50375 non-null  object 
 7   Setting                       60804 non-null  object 
 8   Illnesses                     66713 non-null  object 
 9   Hospitalizations              58155 non-null  float64
 10  Info On Hospitalizations      58480 non-null  object 
 11  Deaths                        58785 non-null  float64
 12  Info On Deaths                58463 non-null  object 
 13  F

The data seems fairly clean for the most part, while some columns seem to have a lot of missing values there is a good chance that it is simply because those columns are about specific outbreak types(foodborne, water, animal). I will need to see if there are any duplicate rows and if there are drop them, combine the Year and Month columns and make the Dtype for the new column DateTime, and correct a couple other Dtypes, but overall I don't expect to have to do a lot of cleaning for this dataset until I get it ready to set up in SQLite.

In [6]:
NORS_df.duplicated().sum()

np.int64(1421)

In [7]:
NORS_df['Primary Mode'].unique()

array(['Water', 'Food', 'Person-to-person', 'Indeterminate/unknown',
       'Animal contact',
       'Environmental contamination other than food/water'], dtype=object)

In [8]:
NORS_df.loc[NORS_df['Primary Mode'] == 'Environmental contamination other than food/water', 'Primary Mode'] = 'Other'
NORS_df['Primary Mode'].unique()
#I do not like using .loc in the slightest, I would have used .replace but my system did not like that

array(['Water', 'Food', 'Person-to-person', 'Indeterminate/unknown',
       'Animal contact', 'Other'], dtype=object)

In [9]:
duplicated_rows = NORS_df[NORS_df.duplicated()]
print(duplicated_rows)

       Year  Month           State           Primary Mode           Etiology  \
521    1983      7       Minnesota                  Water  Norovirus unknown   
1037   1998      1            Ohio                   Food                NaN   
1049   1998      1      Washington                   Food                NaN   
1054   1998      1            Ohio                   Food                NaN   
1101   1998      1         Florida                   Food                NaN   
...     ...    ...             ...                    ...                ...   
66395  2023     11  South Carolina       Person-to-person  Norovirus unknown   
66473  2023     12       Wisconsin       Person-to-person  Norovirus unknown   
66532  2023     12            Iowa  Indeterminate/unknown                NaN   
66660  2023     12       Minnesota       Person-to-person  Norovirus unknown   
66661  2023     12       Minnesota       Person-to-person  Norovirus unknown   

      Serotype or Genotype Etiology Sta

In [10]:
NORS_df.drop_duplicates(keep='first', inplace=True)
NORS_df.duplicated().sum()

np.int64(0)

In [11]:
NORS_df['Month'] = NORS_df['Month'].astype('str').replace({'1': '01', '2': '02', '3': '03', '4': '04', '5': '05', '6': '06', '7': '07', '8': '08', '9': '09'})
NORS_df.head()

,Year,Month,State,Primary Mode,Etiology,Serotype or Genotype,Etiology Status,Setting,Illnesses,Hospitalizations,Info On Hospitalizations,Deaths,Info On Deaths,Food Vehicle,Food Contaminated Ingredient,IFSAC Category,Water Exposure,Water Type,Animal Type
0,1971,02,California,Water,Copper,NaN,Confirmed,Restaurant,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
1,1971,06,Arkansas,Water,Hepatitis A,NaN,Confirmed,Store,98,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Other,NaN
2,1971,06,Missouri,Water,Unknown,NaN,Suspected,Subdivision/Neighborhood,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
3,1971,06,Alabama,Water,Selenium,NaN,Confirmed,Unknown,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Individual/Private,NaN
4,1971,06,Vermont,Water,Unknown,NaN,Suspected,Community/municipality,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN


In [12]:
NORS_df['Date'] = NORS_df['Year'].astype('str') + NORS_df['Month'].astype('str')
NORS_column_to_move = NORS_df.pop('Date')
NORS_df.insert(0, 'Date', NORS_column_to_move)
NORS_df.drop(columns=['Year', 'Month'], inplace= True)
NORS_df.head()

,Date,State,Primary Mode,Etiology,Serotype or Genotype,Etiology Status,Setting,Illnesses,Hospitalizations,Info On Hospitalizations,Deaths,Info On Deaths,Food Vehicle,Food Contaminated Ingredient,IFSAC Category,Water Exposure,Water Type,Animal Type
0,197102,California,Water,Copper,NaN,Confirmed,Restaurant,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
1,197106,Arkansas,Water,Hepatitis A,NaN,Confirmed,Store,98,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Other,NaN
2,197106,Missouri,Water,Unknown,NaN,Suspected,Subdivision/Neighborhood,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
3,197106,Alabama,Water,Selenium,NaN,Confirmed,Unknown,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Individual/Private,NaN
4,197106,Vermont,Water,Unknown,NaN,Suspected,Community/municipality,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN


In [13]:
NORS_df['Date'] = pd.to_datetime(NORS_df['Date'].astype(str) + '01', format='%Y%m%d')
NORS_df['Date'] = NORS_df['Date'].dt.strftime('%Y-%m')
NORS_df.head()

,Date,State,Primary Mode,Etiology,Serotype or Genotype,Etiology Status,Setting,Illnesses,Hospitalizations,Info On Hospitalizations,Deaths,Info On Deaths,Food Vehicle,Food Contaminated Ingredient,IFSAC Category,Water Exposure,Water Type,Animal Type
0,1971-02,California,Water,Copper,NaN,Confirmed,Restaurant,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
1,1971-06,Arkansas,Water,Hepatitis A,NaN,Confirmed,Store,98,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Other,NaN
2,1971-06,Missouri,Water,Unknown,NaN,Suspected,Subdivision/Neighborhood,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
3,1971-06,Alabama,Water,Selenium,NaN,Confirmed,Unknown,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Individual/Private,NaN
4,1971-06,Vermont,Water,Unknown,NaN,Suspected,Community/municipality,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN


In [14]:
NORS_df.drop('Serotype or Genotype', axis=1, inplace=True)
NORS_df.head()
#Looking at the data, there aren't enough rows that don't have null values to make it worth keeping.

,Date,State,Primary Mode,Etiology,Etiology Status,Setting,Illnesses,Hospitalizations,Info On Hospitalizations,Deaths,Info On Deaths,Food Vehicle,Food Contaminated Ingredient,IFSAC Category,Water Exposure,Water Type,Animal Type
0,1971-02,California,Water,Copper,Confirmed,Restaurant,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
1,1971-06,Arkansas,Water,Hepatitis A,Confirmed,Store,98,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Other,NaN
2,1971-06,Missouri,Water,Unknown,Suspected,Subdivision/Neighborhood,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
3,1971-06,Alabama,Water,Selenium,Confirmed,Unknown,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Individual/Private,NaN
4,1971-06,Vermont,Water,Unknown,Suspected,Community/municipality,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN


I have decided to add two columns, an Outbreak ID number and a Primary Mode ID number, so that it is easier to turn this dataframe into a proper relational database.

In [15]:
NORS_df['Outbreak ID'] = range(1, len(NORS_df) + 1)
NORS_column_to_move_2 = NORS_df.pop('Outbreak ID')
NORS_df.insert(0, 'Outbreak ID', NORS_column_to_move_2)
#This will make it easier to keep track of what information is relevant to what outbreak
NORS_df["Primary Mode ID"] = NORS_df["Primary Mode"].astype("category").cat.codes + 1
NORS_column_to_move_3 = NORS_df.pop('Primary Mode ID')
NORS_df.insert(3, 'Primary Mode ID', NORS_column_to_move_3)
#This will (at least in theory) make it easier to sort things into their rlevant tables by Primary Mode
NORS_df.head()

,Outbreak ID,Date,State,Primary Mode ID,Primary Mode,Etiology,Etiology Status,Setting,Illnesses,Hospitalizations,Info On Hospitalizations,Deaths,Info On Deaths,Food Vehicle,Food Contaminated Ingredient,IFSAC Category,Water Exposure,Water Type,Animal Type
0,1,1971-02,California,6,Water,Copper,Confirmed,Restaurant,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
1,2,1971-06,Arkansas,6,Water,Hepatitis A,Confirmed,Store,98,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Other,NaN
2,3,1971-06,Missouri,6,Water,Unknown,Suspected,Subdivision/Neighborhood,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
3,4,1971-06,Alabama,6,Water,Selenium,Confirmed,Unknown,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Individual/Private,NaN
4,5,1971-06,Vermont,6,Water,Unknown,Suspected,Community/municipality,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN


And with that, the inital data cleaning is complete. I may have to do more as I turn everything into SQLite tables and create a relational database, but this is a good starting point.

In [16]:
outbreak_df = NORS_df[['Outbreak ID', 'Date','State', 'Primary Mode ID', 'Etiology','Etiology Status',
                       'Setting','Illnesses','Hospitalizations', 'Info On Hospitalizations', 'Deaths',
                        'Info On Deaths']].drop_duplicates().rename(columns={
    'Outbreak ID': 'outbreak_id',
    'Date':'date',
    'State':'state',
    'Primary Mode ID': 'primary_mode_id',
    'Etiology': 'etiology',
    'Etiology Status':'etiology_status',
    'Setting':'setting',
    'Illnesses':'estimated_total_case_num',
    'Hospitalizations': 'hospitalizations',
    'Info On Hospitalizations': 'hospitalization_info_available',
    'Deaths': 'deaths',
    'Info on Deaths': 'death_info_available'
})

In [17]:
outbreak_df.head()

,outbreak_id,date,state,primary_mode_id,etiology,etiology_status,setting,estimated_total_case_num,hospitalizations,hospitalization_info_available,deaths,Info On Deaths
0,1,1971-02,California,6,Copper,Confirmed,Restaurant,2,NaN,NaN,0.0,NaN
1,2,1971-06,Arkansas,6,Hepatitis A,Confirmed,Store,98,NaN,NaN,0.0,NaN
2,3,1971-06,Missouri,6,Unknown,Suspected,Subdivision/Neighborhood,2,NaN,NaN,0.0,NaN
3,4,1971-06,Alabama,6,Selenium,Confirmed,Unknown,3,NaN,NaN,0.0,NaN
4,5,1971-06,Vermont,6,Unknown,Suspected,Community/municipality,3,NaN,NaN,0.0,NaN


This is going to be the main table used in this dataset, as it has all the information about the outbreaks such as when it happened, where it happened, if it was confirmed or suspected, and more.

In [18]:
mode_df = NORS_df[['Primary Mode ID', 'Primary Mode']].drop_duplicates().rename(columns={
    'Primary Mode ID': 'primary_mode_id',
    'Primary Mode': 'primary_mode'
})
mode_df

,primary_mode_id,primary_mode
0,6,Water
1019,2,Food
15019,5,Person-to-person
15025,3,Indeterminate/unknown
15109,1,Animal contact
16017,4,Other


In [19]:
foodborne_df = (NORS_df[NORS_df['Primary Mode ID'] == 2]
    [['Outbreak ID', 'Primary Mode ID', 'Food Vehicle',
    'Food Contaminated Ingredient', 'IFSAC Category']].drop_duplicates()
    .rename(columns={
        'Outbreak ID': 'outbreak_id',
        'Primary Mode ID': 'primary_mode_id',
        'Food Vehicle': 'food_vehicle',
        'Food Contaminated Ingredient': 'contaminated_ingredient',
        'IFSAC Category': 'IFSAC_category'
    })
)
#I don't think you want to know how long it took me to remember that I could filter like that. It was way too long.
foodborne_df.head()

,outbreak_id,primary_mode_id,food_vehicle,contaminated_ingredient,IFSAC_category
1019,1019,2,"tuna, raw;salmon, raw",NaN,Fish
1020,1020,2,NaN,NaN,NaN
1021,1021,2,custard,NaN,Multiple
1022,1022,2,NaN,NaN,NaN
1023,1023,2,"ice cream, unspecified;cake, unspecified",NaN,Multiple


In [20]:
water_df = (NORS_df[NORS_df['Primary Mode ID'] == 6]
    [['Outbreak ID', 'Primary Mode ID', 'Water Exposure', 
    'Water Type']].drop_duplicates()
    .rename(columns={
        'Outbreak ID': 'outbreak_id',
        'Primary Mode ID': 'primary_mode_id',
        'Water Exposure': 'water_exposure',
        'Water Type': 'water_type'
    })
)
water_df.head()

,outbreak_id,primary_mode_id,water_exposure,water_type
0,1,6,Drinking water,Community
1,2,6,Drinking water,Other
2,3,6,Drinking water,Community
3,4,6,Drinking water,Individual/Private
4,5,6,Drinking water,Community


In [21]:
animal_df = (NORS_df[NORS_df['Primary Mode ID'] == 1]
    [['Outbreak ID', 'Primary Mode ID', 'Animal Type']].drop_duplicates()
    .rename(columns={
        'Outbreak ID': 'outbreak_id',
        'Primary Mode ID': 'primary_mode_id',
        'Animal Type': 'animal_type'
    })
)
animal_df.head()

,outbreak_id,primary_mode_id,animal_type
15109,14916,1,Cattle
15386,15156,1,Cattle
15645,15395,1,Poultry
15702,15452,1,"Bird, not including poultry;Lizard;Reptile (ty..."
15929,15676,1,Poultry


With this, I have all my data broken up into their respective dataframes ready to be implemented in SQLite. While I admittedly don't currently plan to use the water or animal dataframes in this particular project, I decided to go ahead and split them up into their own dataframes in case I ever want to use this dataset in another project in the future.

In [22]:
with sqlite3.connect("../data/NORS.db") as conn:
    outbreak_df.to_sql('outbreaks', conn, index=False, if_exists='replace')
    mode_df.to_sql('mode', conn, index=False, if_exists='replace')
    foodborne_df.to_sql('foodborne', conn, index=False, if_exists='replace')
    water_df.to_sql('water', conn, index=False, if_exists='replace')
    animal_df.to_sql('animal', conn, index=False, if_exists='replace')
    conn.commit()

Now that I have everything ready with the NORS database, it's time for me to switch over to preparing my FDA food recall API. I was able to see examples of the results I would get from the API, and as such I decided to make a function to gather as many of the results as I could from the API while staying withing the limits.

In [ ]:
def fetch_food_recall_data(api_key):
    base_url = 'https://api.fda.gov/food/enforcement.json'
    all_results = []
    limit = 1000 #number of records that match query, max is 1000
    skip = 0 #allows you to skip this number of record that match the search parameter
    max_skip = 25000 #25000 is the largest allowed skip value for this API
    while True:
        params = {
            'api_key': api_key,
            'limit': limit,
            'skip': skip
        }
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"Request failed due to status code {response.status_code}")
            break
        data = response.json()
        if 'results' not in data:
            print("No more results for this API.")
            break
        results = data['results']
        all_results.extend(results)
         #This part is technically not needed but it is nice seeing the progress being made in regards to the data gathering:
        print(f"Fetched {len(results)} records")
        if len(results) < limit or skip == max_skip:
            break
        skip += limit
        time.sleep(0.2) #give short pause to avoid somehow accidentally hitting the 240 request per minute rate limit
    df = pd.json_normalize(all_results)
    return df

While this function was made primarily with this specific API in mind, I can also edit it for use with other APIs for any future projects I make. The purpose of this function is to automate the gathering of the API results so that I can easily convert those results into a csv for cleaning.

In [ ]:
recall_df = fetch_food_recall_data('#Insert your key here!')

Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records
Fetched 1000 records


In [25]:
recall_df.to_csv('../data/FDA_food_recall.csv', index=False)

In [27]:
recall_df

,status,city,state,country,classification,product_type,event_id,recalling_firm,address_1,address_2,...,recall_number,product_description,product_quantity,reason_for_recall,recall_initiation_date,center_classification_date,termination_date,report_date,code_info,more_code_info
0,Terminated,Davie,FL,United States,Class II,Food,75272,Pharmatech LLC,4131 SW 47th Ave Ste 1403,,...,F-0276-2017,"CytoDetox, Hydrolyzed Clinoptilolite Fragments...","1,990 bottles",Recall initiated as a precautionary measure du...,20160808,20161025,20240923,20161102,"UPC No. 632687615989; Lot No. 30661601, Exp. D...",
1,Terminated,Millbrae,CA,United States,Class II,Food,75069,Magic Gourmet Trading Inc,375 Adrian Rd,,...,F-0865-2017,Koi Palace Mini Moon Cake: Single Box - Mini O...,"2 cases (1 pc/bx, 48bx/cs)","Mooncake products, manufactured and distribute...",20160831,20170106,20170111,20170118,"FG-M1MOT-UW Best by Nov 1, 2016.",
2,Terminated,Miami,FL,United States,Class I,Food,69516,"Oasis Brands, Inc",13439 NW 19 LANE,N/A,...,F-0609-2015,Crema GuateLinda (Guatemalan Style Cream) in i...,144 pieces,Virginia State (VDACS) found Listeria monocyto...,20141010,20141202,20170328,20141210,UPC 635349 000390 Best By dates: 07/01/14 thr...,NaN
3,Terminated,Pompano Beach,FL,United States,Class I,Food,62750,FreshPoint South Florida,2300 NW 19th St,,...,F-1922-2012,"Yellow Onion. Product is labeled in part: ""*...",7 cases,FreshPoint South Florida is recalling sliced f...,20120727,20120808,20141008,20120815,Item # 302940.,
4,Terminated,Rancho Dominguez,CA,United States,Class III,Food,85253,"Organic By Nature, Inc.",2610 Homestead Pl,N/A,...,F-0904-2020,Pure Planet Organic Parasite Cleanse; Net Wt....,xx,Firm was notified by supplier that Organic Gro...,20200224,20200413,20210202,20200401,Lot codes: 72746,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25995,Terminated,Boston,MA,United States,Class III,Food,77570,Channel Fish Processing Co. Inc,18 Food Mart Rd,N/A,...,F-2918-2017,Ocean's Best - Raw Breaded Fish Squares ( 3OZ ...,"1,055,006 pounds (total distributed for all pr...",Products recalled due to undeclared allergen -...,20170613,20170725,20170912,20170802,"3OZ squares lot #'s - 23567, 22748, 23042 4OZ...",NaN
25996,Terminated,Massillon,OH,United States,Class II,Food,80992,"Shearers Foods, Llc",100 Lincoln Way E,N/A,...,F-1966-2018,White Corn Tortilla Chips packaged under the f...,"6,750 cases total",Corn Chip products are being recalled due to p...,20180907,20180917,20200617,20180926,"Bag Code Date: 1. 12/31/2018, 2. 12/31/...",NaN
25997,Terminated,Los Angeles,CA,United States,Class II,Food,70776,A I Foods Corporation,1700 N Soto St,N/A,...,F-1935-2015,"Cheesetix (Cheese Lumpia) Pimiento (12.6 oz), ...",20 units,AI Foods is recalling egg roll skins due to un...,20150318,20150409,20150724,20150415,All,NaN
25998,Terminated,Eden Prairie,MN,United States,Class I,Food,92573,SunOpta Grains and Foods Inc.,7078 Shady Oak Rd,N/A,...,F-1256-2023,Pineapple Chunks packaged under the following ...,"22995 cases + 232,300 lbs. bulk",Fruit products have the potential to be contam...,20230616,20230718,20240207,20230726,1. Lot / Best By: FED303132 / 01/31/2025; ...,
